In [42]:
import os
import xmltodict
import pandas as pd
import random
from pprint import pprint
from utils.tui import Progress

BASE_DIR = '../data/corpora/trainset'
TEST_PERCENT = 0.2

In [7]:
count = 0
for product in os.listdir(BASE_DIR):
    corpus = {}
    
    product_path = '{}/{}'.format(BASE_DIR, product)
    for rank in os.listdir(product_path):  
        rank_path = '{}/{}'.format(product_path, rank)
        fls = os.listdir(rank_path)
        fls = [ x for x in fls if '.xml' in x ]
        count += len(fls)

In [9]:
corpora_kabum = {}
errors = []
P = Progress(count, '')
for product in os.listdir(BASE_DIR):
    corpus = {}
    
    product_path = '{}/{}'.format(BASE_DIR, product)
    for rank in os.listdir(product_path):  
        rank_path = '{}/{}'.format(product_path, rank)
        fls = os.listdir(rank_path)
        fls = [ x for x in fls if '.xml' in x ]
        
        reviews = []
        for fl_name in fls:
            fl_path = '{}/{}'.format(rank_path, fl_name)
            with open(fl_path) as fl_:
                try:
                    reviews.append(xmltodict.parse(fl_.read())['review'])
                except:
                    errors.append(fl_path)
            P.progressStep()
        corpus[rank] = reviews
    corpora_kabum[product] = corpus

In [51]:
dataset = dict(test=[], category=[], score=[], review=[])

for cat in corpora_kabum.keys():
    for stars in corpora_kabum[cat].keys():
        for review in corpora_kabum[cat][stars]:
            dataset['test'].append(random.choices([0,1], weights=[0.8, 0.2],k=1)[0])
            dataset['category'].append(review['category']['@value'])
            dataset['score'].append(float(review['stars']['@value']))
            dataset['review'].append(review['opinion'])
dataframe = pd.DataFrame(data=dataset)

In [52]:
import pickle
import gzip

b_ = pickle.dumps(dataframe)
with gzip.open('dataframe_kabum.pkl.gz', 'wb') as f:
    f.write(b_)

In [56]:
train_x = dataframe.loc[dataframe.test == 1].review.values.tolist()
train_y = dataframe.loc[dataframe.test == 1].score.values.tolist()

from sklearn.feature_extraction.text import TfidfVectorizer
transformer = TfidfVectorizer()
transformer.fit(train_reviews)
X = transformer.transform(train_reviews)

ModuleNotFoundError: No module named 'sklearn'